In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import matplotlib as plt
from datetime import datetime, timedelta
import re
from autogluon.tabular import TabularPredictor
from sklearn.metrics import mean_absolute_error

d:\Renewables\towie\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data_fpath =  "training_dataset.parquet"
submission_data_fpath =  "submission_dataset.parquet"

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
input_df = pd.read_parquet(train_data_fpath)
test_df = pd.read_parquet(submission_data_fpath)
input_df.head(3)
input_df = input_df[~input_df.target.isna()]

In [5]:
input_df["hour"] = input_df["TimeStamp_StartFormat"].dt.hour
test_df["hour"] = test_df["TimeStamp_StartFormat"].dt.hour

In [6]:
len(input_df)

207924

In [7]:
input_df.columns

Index(['TimeStamp_StartFormat', 'wtc_AcWindSp_mean;1', 'wtc_AcWindSp_mean;2',
       'wtc_AcWindSp_mean;3', 'wtc_AcWindSp_mean;4', 'wtc_AcWindSp_mean;5',
       'wtc_AcWindSp_mean;7', 'wtc_AcWindSp_min;1', 'wtc_AcWindSp_min;2',
       'wtc_AcWindSp_min;3',
       ...
       'ERA5_cloud_cover', 'ERA5_wind_speed_10m', 'ERA5_wind_speed_100m',
       'ERA5_wind_direction_10m', 'ERA5_wind_direction_100m',
       'ERA5_wind_gusts_10m', 'id', 'is_valid', 'target', 'hour'],
      dtype='object', length=190)

In [8]:
input_df.head(3)

,TimeStamp_StartFormat,wtc_AcWindSp_mean;1,wtc_AcWindSp_mean;2,wtc_AcWindSp_mean;3,wtc_AcWindSp_mean;4,wtc_AcWindSp_mean;5,wtc_AcWindSp_mean;7,wtc_AcWindSp_min;1,wtc_AcWindSp_min;2,wtc_AcWindSp_min;3,wtc_AcWindSp_min;4,wtc_AcWindSp_min;5,wtc_AcWindSp_min;7,wtc_AcWindSp_max;1,wtc_AcWindSp_max;2,wtc_AcWindSp_max;3,wtc_AcWindSp_max;4,wtc_AcWindSp_max;5,wtc_AcWindSp_max;7,wtc_AcWindSp_stddev;1,wtc_AcWindSp_stddev;2,wtc_AcWindSp_stddev;3,wtc_AcWindSp_stddev;4,wtc_AcWindSp_stddev;5,wtc_AcWindSp_stddev;7,wtc_ScYawPos_mean;1,wtc_ScYawPos_mean;2,wtc_ScYawPos_mean;3,wtc_ScYawPos_mean;4,wtc_ScYawPos_mean;5,wtc_ScYawPos_mean;7,wtc_ScYawPos_min;1,wtc_ScYawPos_min;2,wtc_ScYawPos_min;3,wtc_ScYawPos_min;4,wtc_ScYawPos_min;5,wtc_ScYawPos_min;7,wtc_ScYawPos_max;1,wtc_ScYawPos_max;2,wtc_ScYawPos_max;3,wtc_ScYawPos_max;4,wtc_ScYawPos_max;5,wtc_ScYawPos_max;7,wtc_ScYawPos_stddev;1,wtc_ScYawPos_stddev;2,wtc_ScYawPos_stddev;3,wtc_ScYawPos_stddev;4,wtc_ScYawPos_stddev;5,wtc_ScYawPos_stddev;7,wtc_NacelPos_mean;1,wtc_NacelPos_mean;2,wtc_NacelPos_mean;3,wtc_NacelPos_mean;4,wtc_NacelPos_mean;5,wtc_NacelPos_mean;7,wtc_NacelPos_min;1,wtc_NacelPos_min;2,wtc_NacelPos_min;3,wtc_NacelPos_min;4,wtc_NacelPos_min;5,wtc_NacelPos_min;7,wtc_NacelPos_max;1,wtc_NacelPos_max;2,wtc_NacelPos_max;3,wtc_NacelPos_max;4,wtc_NacelPos_max;5,wtc_NacelPos_max;7,wtc_GenRpm_mean;1,wtc_GenRpm_mean;2,wtc_GenRpm_mean;3,wtc_GenRpm_mean;4,wtc_GenRpm_mean;5,wtc_GenRpm_mean;7,wtc_GenRpm_min;1,wtc_GenRpm_min;2,wtc_GenRpm_min;3,wtc_GenRpm_min;4,wtc_GenRpm_min;5,wtc_GenRpm_min;7,wtc_GenRpm_max;1,wtc_GenRpm_max;2,wtc_GenRpm_max;3,wtc_GenRpm_max;4,wtc_GenRpm_max;5,wtc_GenRpm_max;7,wtc_GenRpm_stddev;1,wtc_GenRpm_stddev;2,wtc_GenRpm_stddev;3,wtc_GenRpm_stddev;4,wtc_GenRpm_stddev;5,wtc_GenRpm_stddev;7,wtc_PitcPosA_mean;1,wtc_PitcPosA_mean;2,wtc_PitcPosA_mean;3,wtc_PitcPosA_mean;4,wtc_PitcPosA_mean;5,wtc_PitcPosA_mean;7,wtc_PitcPosA_min;1,wtc_PitcPosA_min;2,wtc_PitcPosA_min;3,wtc_PitcPosA_min;4,wtc_PitcPosA_min;5,wtc_PitcPosA_min;7,wtc_PitcPosA_max;1,wtc_PitcPosA_max;2,wtc_PitcPosA_max;3,wtc_PitcPosA_max;4,wtc_PitcPosA_max;5,wtc_PitcPosA_max;7,wtc_PitcPosA_stddev;1,wtc_PitcPosA_stddev;2,wtc_PitcPosA_stddev;3,wtc_PitcPosA_stddev;4,wtc_PitcPosA_stddev;5,wtc_PitcPosA_stddev;7,wtc_PitcPosB_mean;1,wtc_PitcPosB_mean;2,wtc_PitcPosB_mean;3,wtc_PitcPosB_mean;4,wtc_PitcPosB_mean;5,wtc_PitcPosB_mean;7,wtc_PitcPosC_mean;1,wtc_PitcPosC_mean;2,wtc_PitcPosC_mean;3,wtc_PitcPosC_mean;4,wtc_PitcPosC_mean;5,wtc_PitcPosC_mean;7,wtc_PowerRef_endvalue;1,wtc_PowerRef_endvalue;2,wtc_PowerRef_endvalue;3,wtc_PowerRef_endvalue;4,wtc_PowerRef_endvalue;5,wtc_PowerRef_endvalue;7,wtc_ScReToOp_timeon;1,wtc_ScReToOp_timeon;2,wtc_ScReToOp_timeon;3,wtc_ScReToOp_timeon;4,wtc_ScReToOp_timeon;5,wtc_ScReToOp_timeon;7,wtc_ActPower_mean;1,wtc_ActPower_mean;2,wtc_ActPower_mean;3,wtc_ActPower_mean;4,wtc_ActPower_mean;5,wtc_ActPower_mean;7,wtc_ActPower_min;1,wtc_ActPower_min;2,wtc_ActPower_min;3,wtc_ActPower_min;4,wtc_ActPower_min;5,wtc_ActPower_min;7,wtc_ActPower_max;1,wtc_ActPower_max;2,wtc_ActPower_max;3,wtc_ActPower_max;4,wtc_ActPower_max;5,wtc_ActPower_max;7,wtc_ActPower_stddev;1,wtc_ActPower_stddev;2,wtc_ActPower_stddev;3,wtc_ActPower_stddev;4,wtc_ActPower_stddev;5,wtc_ActPower_stddev;7,wtc_AmbieTmp_mean;1,wtc_AmbieTmp_mean;2,wtc_AmbieTmp_mean;3,wtc_AmbieTmp_mean;4,wtc_AmbieTmp_mean;5,wtc_AmbieTmp_mean;7,ShutdownDuration;1,ShutdownDuration;2,ShutdownDuration;3,ShutdownDuration;4,ShutdownDuration;5,ShutdownDuration;7,ERA5_temperature_2m,ERA5_relative_humidity_2m,ERA5_dew_point_2m,ERA5_precipitation,ERA5_surface_pressure,ERA5_cloud_cover,ERA5_wind_speed_10m,ERA5_wind_speed_100m,ERA5_wind_direction_10m,ERA5_wind_direction_100m,ERA5_wind_gusts_10m,id,is_valid,target,hour
0,2016-01-01 00:00:00+00:00,6.085917,5.830675,6.613091,6.998510,7.645727,7.040790,2.0,1.1,1.6,2.4,4.3,3.9,10.0,10.0,11.0,10.6,11.8,10.5,1.484347,1.586648,1.571053,1.228338,1.096910,1.317204,83.754387,69.120506,-122.800903,-113.638397,-118.159103,-106.894501,79.500000,66.

In [9]:
input_df.tail(3)

,TimeStamp_StartFormat,wtc_AcWindSp_mean;1,wtc_AcWindSp_mean;2,wtc_AcWindSp_mean;3,wtc_AcWindSp_mean;4,wtc_AcWindSp_mean;5,wtc_AcWindSp_mean;7,wtc_AcWindSp_min;1,wtc_AcWindSp_min;2,wtc_AcWindSp_min;3,wtc_AcWindSp_min;4,wtc_AcWindSp_min;5,wtc_AcWindSp_min;7,wtc_AcWindSp_max;1,wtc_AcWindSp_max;2,wtc_AcWindSp_max;3,wtc_AcWindSp_max;4,wtc_AcWindSp_max;5,wtc_AcWindSp_max;7,wtc_AcWindSp_stddev;1,wtc_AcWindSp_stddev;2,wtc_AcWindSp_stddev;3,wtc_AcWindSp_stddev;4,wtc_AcWindSp_stddev;5,wtc_AcWindSp_stddev;7,wtc_ScYawPos_mean;1,wtc_ScYawPos_mean;2,wtc_ScYawPos_mean;3,wtc_ScYawPos_mean;4,wtc_ScYawPos_mean;5,wtc_ScYawPos_mean;7,wtc_ScYawPos_min;1,wtc_ScYawPos_min;2,wtc_ScYawPos_min;3,wtc_ScYawPos_min;4,wtc_ScYawPos_min;5,wtc_ScYawPos_min;7,wtc_ScYawPos_max;1,wtc_ScYawPos_max;2,wtc_ScYawPos_max;3,wtc_ScYawPos_max;4,wtc_ScYawPos_max;5,wtc_ScYawPos_max;7,wtc_ScYawPos_stddev;1,wtc_ScYawPos_stddev;2,wtc_ScYawPos_stddev;3,wtc_ScYawPos_stddev;4,wtc_ScYawPos_stddev;5,wtc_ScYawPos_stddev;7,wtc_NacelPos_mean;1,wtc_NacelPos_mean;2,wtc_NacelPos_mean;3,wtc_NacelPos_mean;4,wtc_NacelPos_mean;5,wtc_NacelPos_mean;7,wtc_NacelPos_min;1,wtc_NacelPos_min;2,wtc_NacelPos_min;3,wtc_NacelPos_min;4,wtc_NacelPos_min;5,wtc_NacelPos_min;7,wtc_NacelPos_max;1,wtc_NacelPos_max;2,wtc_NacelPos_max;3,wtc_NacelPos_max;4,wtc_NacelPos_max;5,wtc_NacelPos_max;7,wtc_GenRpm_mean;1,wtc_GenRpm_mean;2,wtc_GenRpm_mean;3,wtc_GenRpm_mean;4,wtc_GenRpm_mean;5,wtc_GenRpm_mean;7,wtc_GenRpm_min;1,wtc_GenRpm_min;2,wtc_GenRpm_min;3,wtc_GenRpm_min;4,wtc_GenRpm_min;5,wtc_GenRpm_min;7,wtc_GenRpm_max;1,wtc_GenRpm_max;2,wtc_GenRpm_max;3,wtc_GenRpm_max;4,wtc_GenRpm_max;5,wtc_GenRpm_max;7,wtc_GenRpm_stddev;1,wtc_GenRpm_stddev;2,wtc_GenRpm_stddev;3,wtc_GenRpm_stddev;4,wtc_GenRpm_stddev;5,wtc_GenRpm_stddev;7,wtc_PitcPosA_mean;1,wtc_PitcPosA_mean;2,wtc_PitcPosA_mean;3,wtc_PitcPosA_mean;4,wtc_PitcPosA_mean;5,wtc_PitcPosA_mean;7,wtc_PitcPosA_min;1,wtc_PitcPosA_min;2,wtc_PitcPosA_min;3,wtc_PitcPosA_min;4,wtc_PitcPosA_min;5,wtc_PitcPosA_min;7,wtc_PitcPosA_max;1,wtc_PitcPosA_max;2,wtc_PitcPosA_max;3,wtc_PitcPosA_max;4,wtc_PitcPosA_max;5,wtc_PitcPosA_max;7,wtc_PitcPosA_stddev;1,wtc_PitcPosA_stddev;2,wtc_PitcPosA_stddev;3,wtc_PitcPosA_stddev;4,wtc_PitcPosA_stddev;5,wtc_PitcPosA_stddev;7,wtc_PitcPosB_mean;1,wtc_PitcPosB_mean;2,wtc_PitcPosB_mean;3,wtc_PitcPosB_mean;4,wtc_PitcPosB_mean;5,wtc_PitcPosB_mean;7,wtc_PitcPosC_mean;1,wtc_PitcPosC_mean;2,wtc_PitcPosC_mean;3,wtc_PitcPosC_mean;4,wtc_PitcPosC_mean;5,wtc_PitcPosC_mean;7,wtc_PowerRef_endvalue;1,wtc_PowerRef_endvalue;2,wtc_PowerRef_endvalue;3,wtc_PowerRef_endvalue;4,wtc_PowerRef_endvalue;5,wtc_PowerRef_endvalue;7,wtc_ScReToOp_timeon;1,wtc_ScReToOp_timeon;2,wtc_ScReToOp_timeon;3,wtc_ScReToOp_timeon;4,wtc_ScReToOp_timeon;5,wtc_ScReToOp_timeon;7,wtc_ActPower_mean;1,wtc_ActPower_mean;2,wtc_ActPower_mean;3,wtc_ActPower_mean;4,wtc_ActPower_mean;5,wtc_ActPower_mean;7,wtc_ActPower_min;1,wtc_ActPower_min;2,wtc_ActPower_min;3,wtc_ActPower_min;4,wtc_ActPower_min;5,wtc_ActPower_min;7,wtc_ActPower_max;1,wtc_ActPower_max;2,wtc_ActPower_max;3,wtc_ActPower_max;4,wtc_ActPower_max;5,wtc_ActPower_max;7,wtc_ActPower_stddev;1,wtc_ActPower_stddev;2,wtc_ActPower_stddev;3,wtc_ActPower_stddev;4,wtc_ActPower_stddev;5,wtc_ActPower_stddev;7,wtc_AmbieTmp_mean;1,wtc_AmbieTmp_mean;2,wtc_AmbieTmp_mean;3,wtc_AmbieTmp_mean;4,wtc_AmbieTmp_mean;5,wtc_AmbieTmp_mean;7,ShutdownDuration;1,ShutdownDuration;2,ShutdownDuration;3,ShutdownDuration;4,ShutdownDuration;5,ShutdownDuration;7,ERA5_temperature_2m,ERA5_relative_humidity_2m,ERA5_dew_point_2m,ERA5_precipitation,ERA5_surface_pressure,ERA5_cloud_cover,ERA5_wind_speed_10m,ERA5_wind_speed_100m,ERA5_wind_direction_10m,ERA5_wind_direction_100m,ERA5_wind_gusts_10m,id,is_valid,target,hour
210381,2019-12-31 23:30:00+00:00,8.292965,7.242282,7.387715,8.898215,7.778623,7.802809,4.98,4.98,4.49,5.49,5.18,5.28,10.79,9.54,9.85,11.86,10.55,9.75,0.972607,0.881621,0.918921,0.909908,0.892530,0.779341,-96.173378,-124.966599,-116.936699,-113.882103,-133.844696,-102.936

In [10]:
def rel_dir(df):
    for turbine_num in [2, 3, 4, 5, 7]:
        yaw_col = f'wtc_ScYawPos_mean;{turbine_num}'
        wind_dir_col = 'ERA5_wind_direction_10m'  # or 100m depending on your needs
        
        # Calculate relative angle
        df[f'relative_angle_{turbine_num}'] = (df[wind_dir_col] - df[yaw_col]).abs()
        
        # Normalize to [-180, 180]
        df[f'relative_angle_{turbine_num}'] = df[f'relative_angle_{turbine_num}'].apply(
            lambda x: x if x <= 180 else 360 - x
        )
    return df

In [11]:
def day_feature_adder(df):
    day_of_year = df['TimeStamp_StartFormat'].dt.dayofyear
    df['daylight_hours'] = 8 + 4 * np.sin(2 * np.pi * (day_of_year - 80) / 365.25)
    df['is_peak_wind'] = np.where(df['TimeStamp_StartFormat'].dt.month.isin([12, 1, 2]),1,0)



    df['minute_sin'] = np.sin(2 * np.pi * df['TimeStamp_StartFormat'].dt.minute / 60)
    df['minute_cos'] = np.cos(2 * np.pi * df['TimeStamp_StartFormat'].dt.minute / 60)

    df['hour_sin'] = np.sin(2 * np.pi * df['TimeStamp_StartFormat'].dt.hour / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['TimeStamp_StartFormat'].dt.hour / 24)

    df['dayofyear_sin'] = np.sin(2 * np.pi * df['TimeStamp_StartFormat'].dt.dayofyear / 365)
    df['dayofyear_cos'] = np.cos(2 * np.pi * df['TimeStamp_StartFormat'].dt.dayofyear / 365)

    df['month_sin'] = np.sin(2 * np.pi * df['TimeStamp_StartFormat'].dt.month / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['TimeStamp_StartFormat'].dt.month / 12)
    return df

In [12]:
def add_lagged_wind_speeds(df, timestamp_col='TimeStamp_StartFormat', turbine_prefix=';'):
    """
    Add lagged wind speed features for each turbine (15min and 1hr lags)
    
    Args:
        df (pd.DataFrame): Must contain:
            - timestamp_col: DateTime column
            - f'Wind_speed_{turbine_prefix}N' for each turbine
        turbine_prefix (str): Prefix used in turbine column names
        
    Returns:
        pd.DataFrame: Original df + new lagged wind speed columns
    """
    # Make sure we're working with a copy
    df = df.copy()
    
    # Sort by time to ensure proper shifting
    df = df.sort_values(timestamp_col).reset_index(drop=True)
    
    # Time deltas for lags
    lag_30min = timedelta(minutes=10)
    lag_1hr = timedelta(minutes=60)
    
    for turbine_num in [2,3,4,5,7]:#,3,4]:  # For turbines 2,3,4,5,7 (6 being predicted)
        wind_col = f'wtc_ActPower_mean{turbine_prefix}{turbine_num}'#'wtc_GenRpm_mean
        
        # Create temporary DataFrame for merging
        lagged = df[[timestamp_col, wind_col]].copy()
        
        # Calculate 30min lag
        lagged['temp_30min'] = lagged[timestamp_col] + lag_30min
        lagged = lagged.rename(columns={wind_col: f'{wind_col}_lag30min'})
        df = pd.merge_asof(
            df.sort_values(timestamp_col),
            lagged[['temp_30min', f'{wind_col}_lag30min']].sort_values('temp_30min'),
            left_on=timestamp_col,
            right_on='temp_30min',
            direction='backward'
        ).drop(columns='temp_30min')
        
        # Calculate 1hr lag
        lagged['temp_1hr'] = lagged[timestamp_col] + lag_1hr
        lagged = lagged.rename(columns={f'{wind_col}_lag30min': f'{wind_col}_lag1hr'})
        df = pd.merge_asof(
            df.sort_values(timestamp_col),
            lagged[['temp_1hr', f'{wind_col}_lag1hr']].sort_values('temp_1hr'),
            left_on=timestamp_col,
            right_on='temp_1hr',
            direction='backward'
        ).drop(columns='temp_1hr')
    
    return df

In [13]:
def add_lagged_act_power(df, timestamp_col='TimeStamp_StartFormat', 
                         lags=[(10, '10min'),(60, '60min')], 
                         turbine_prefix=';'):
    """
    Add lagged active power features for each turbine
    
    Args:
        df (pd.DataFrame): Must contain:
            - timestamp_col: DateTime column
            - Columns containing 'ActPower' in their names
        lags (list of tuples): List of (minutes, suffix_name) for each lag
        turbine_prefix (str): Prefix used in turbine column names
        
    Returns:
        pd.DataFrame: Original df + new lagged active power columns
    """
    # Make sure we're working with a copy
    df = df.copy()
    
    # Sort by time to ensure proper shifting
    df = df.sort_values(timestamp_col).reset_index(drop=True)
    
    # Find all ActPower columns
    act_power_cols = [col for col in df.columns if "wtc_Ac" in col]#'ActPower''wtc_Ac'
    
    for act_power_col in act_power_cols:
        for minutes, suffix in lags:
            # Create temporary DataFrame for merging
            lagged = df[[timestamp_col, act_power_col]].copy()
            lag_delta = timedelta(minutes=minutes)
            
            # Calculate lag
            lagged['temp_time'] = lagged[timestamp_col] + lag_delta
            lagged = lagged.rename(columns={act_power_col: f'{act_power_col}_lag{suffix}'})
            
            df = pd.merge_asof(
                df.sort_values(timestamp_col),
                lagged[['temp_time', f'{act_power_col}_lag{suffix}']].sort_values('temp_time'),
                left_on=timestamp_col,
                right_on='temp_time',
                direction='backward'
            ).drop(columns='temp_time')
    
    return df

In [14]:
def add_trig_transforms(df, substrings=('Pos', 'direction'), radians=True):
    """
    Adds sin and cos transformations for columns containing any of the specified substrings.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Input DataFrame
    substrings : tuple of str (default=('Pos', 'direction'))
        Substrings to identify columns to transform
    radians : bool (default=True)
        If False, converts values from degrees to radians first
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame with new sin/cos columns added
    """
    df_transformed = df.copy()
    
    for col in df.columns:
        if any(sub in col for sub in substrings):
            values = df[col]
            
            if not radians:
                values = np.radians(values)
                
            df_transformed[f'{col}_sin'] = np.sin(values)
            df_transformed[f'{col}_cos'] = np.cos(values)
    
    return df_transformed

In [15]:
from scipy.signal import periodogram

# Dominant frequency in last 6 hours (captures oscillations)
def get_dominant_freq(series):
    f, Pxx = periodogram(series.dropna())
    return f[np.argmax(Pxx)]


In [16]:
def new_features(df):
    df["TimeStamp_StartFormat"] = pd.to_datetime(df['TimeStamp_StartFormat'])
    df["wind_speed2_rolling_15min"] = df[['TimeStamp_StartFormat','wtc_AcWindSp_mean;2']].groupby('wtc_AcWindSp_mean;2',as_index=False).rolling('30T', on='TimeStamp_StartFormat').mean().rename(columns={"wtc_AcWindSp_mean;2":"wind_speed_rolling_15min"})
    df["wind_speed2_rolling_60min"]= df[['TimeStamp_StartFormat','wtc_AcWindSp_mean;2']].groupby('wtc_AcWindSp_mean;2',as_index=False).rolling('1H', on='TimeStamp_StartFormat').mean().rename(columns={"wtc_AcWindSp_mean;2":"wind_speed_rolling_60min"})
    #df["wind_speed3_rolling_15min"] = df[['TimeStamp_StartFormat','wtc_AcWindSp_mean;3']].groupby('wtc_AcWindSp_mean;3',as_index=False).rolling('30T', on='TimeStamp_StartFormat').mean().rename(columns={"wtc_AcWindSp_mean;3":"wind_speed_rolling_15min"})
    #df["wind_speed3_rolling_60min"]= df[['TimeStamp_StartFormat','wtc_AcWindSp_mean;3']].groupby('wtc_AcWindSp_mean;3',as_index=False).rolling('1H', on='TimeStamp_StartFormat').mean().rename(columns={"wtc_AcWindSp_mean;3":"wind_speed_rolling_60min"})
    #df["wind_speed4_rolling_15min"] = df[['TimeStamp_StartFormat','wtc_AcWindSp_mean;4']].groupby('wtc_AcWindSp_mean;4',as_index=False).rolling('30T', on='TimeStamp_StartFormat').mean().rename(columns={"wtc_AcWindSp_mean;4":"wind_speed_rolling_15min"})
    #df["wind_speed4_rolling_60min"]= df[['TimeStamp_StartFormat','wtc_AcWindSp_mean;4']].groupby('wtc_AcWindSp_mean;4',as_index=False).rolling('1H', on='TimeStamp_StartFormat').mean().rename(columns={"wtc_AcWindSp_mean;4":"wind_speed_rolling_60min"})
    #df["wind_speed5_rolling_15min"] = df[['TimeStamp_StartFormat','wtc_AcWindSp_mean;5']].groupby('wtc_AcWindSp_mean;5',as_index=False).rolling('30T', on='TimeStamp_StartFormat').mean().rename(columns={"wtc_AcWindSp_mean;5":"wind_speed_rolling_15min"})
    #df["wind_speed5_rolling_60min"]= df[['TimeStamp_StartFormat','wtc_AcWindSp_mean;5']].groupby('wtc_AcWindSp_mean;5',as_index=False).rolling('1H', on='TimeStamp_StartFormat').mean().rename(columns={"wtc_AcWindSp_mean;5":"wind_speed_rolling_60min"})

    # For neighboring turbines (e.g., Kelmarsh 2)
    #df["neighbor_rolling_30min"] = df[['TimeStamp_StartFormat','wtc_AcWindSp_mean;3']].groupby('wtc_AcWindSp_mean;3',as_index=False).rolling('30T', on='TimeStamp_StartFormat').mean().rename(columns={"wtc_AcWindSp_mean;3":"neighbor_rolling_30min"})
    # Rate of change (derivative)
    df['wind_speed2_diff_10min'] = df['wtc_AcWindSp_mean;2'].diff(periods=6)  # For 10-min data
    df['wind_speed2_pct_change_30min'] = df['wtc_AcWindSp_mean;2'].pct_change(periods=3)
    df['wind_dir2_stability_1hr'] = df[['TimeStamp_StartFormat','wtc_AcWindSp_mean;2']].groupby('wtc_AcWindSp_mean;2',as_index=False).rolling('1H', on='TimeStamp_StartFormat').std().rename(columns={"wtc_AcWindSp_mean;2":'wind_dir_stability_1hr'})
    #df['wind_speed3_diff_10min'] = df['wtc_AcWindSp_mean;3'].diff(periods=6)  # For 10-min data
    #df['wind_speed3_pct_change_30min'] = df['wtc_AcWindSp_mean;3'].pct_change(periods=3)
    #df['wind_dir3_stability_1hr'] = df[['TimeStamp_StartFormat','wtc_AcWindSp_mean;3']].groupby('wtc_AcWindSp_mean;3',as_index=False).rolling('1H', on='TimeStamp_StartFormat').std().rename(columns={"wtc_AcWindSp_mean;3":'wind_dir_stability_1hr'})
    #df['wind_speed4_diff_10min'] = df['wtc_AcWindSp_mean;4'].diff(periods=6)  # For 10-min data
    #df['wind_speed4_pct_change_30min'] = df['wtc_AcWindSp_mean;4'].pct_change(periods=3)
    #df['wind_dir4_stability_1hr'] = df[['TimeStamp_StartFormat','wtc_AcWindSp_mean;4']].groupby('wtc_AcWindSp_mean;4',as_index=False).rolling('1H', on='TimeStamp_StartFormat').std().rename(columns={"wtc_AcWindSp_mean;4":'wind_dir_stability_1hr'})
    #df['wind_speed5_diff_10min'] = df['wtc_AcWindSp_mean;5'].diff(periods=6)  # For 10-min data
    #df['wind_speed5_pct_change_30min'] = df['wtc_AcWindSp_mean;5'].pct_change(periods=3)
    #df['wind_dir5_stability_1hr'] = df[['TimeStamp_StartFormat','wtc_AcWindSp_mean;5']].groupby('wtc_AcWindSp_mean;5',as_index=False).rolling('1H', on='TimeStamp_StartFormat').std().rename(columns={"wtc_AcWindSp_mean;5":'wind_dir_stability_1hr'})
    #df['wind_speed6_diff_10min'] = df['wtc_AcWindSp_mean;7'].diff(periods=6)  # For 10-min data
    #df['wind_speed6_pct_change_30min'] = df['wtc_AcWindSp_mean;7'].pct_change(periods=3)
    #df['wind_dir6_stability_1hr'] = df[['TimeStamp_StartFormat','wtc_AcWindSp_mean;7']].groupby('wtc_AcWindSp_mean;7',as_index=False).rolling('1H', on='TimeStamp_StartFormat').std().rename(columns={"wtc_AcWindSp_mean;7":'wind_dir_stability_1hr'})


# Gust detection (1.5x rolling mean)
    df['gust_flag'] = (df['wtc_AcWindSp_mean;2'] > 1.5 * df['wind_speed2_rolling_15min']).astype(int)
    df['upwind_turbine_lag10min'] = df['wtc_AcWindSp_mean;3'].shift(periods=1)  # Assuming 10-min data

# Cross-turbine rolling correlations
    #df['rolling_corr_1hr'] = df[['wtc_AcWindSp_mean;3','wtc_AcWindSp_mean;2']].corr()["wtc_AcWindSp_mean;2"][0]


    df['dominant_freq2_2hr'] = df[['TimeStamp_StartFormat','wtc_AcWindSp_mean;2']].groupby('wtc_AcWindSp_mean;2',as_index=False).rolling('2H', on='TimeStamp_StartFormat').apply(get_dominant_freq)
    #df['dominant_freq3_2hr'] = df[['TimeStamp_StartFormat','wtc_AcWindSp_mean;3']].groupby('wtc_AcWindSp_mean;3',as_index=False).rolling('2H', on='TimeStamp_StartFormat').apply(get_dominant_freq)
    #df['dominant_freq4_2hr'] = df[['TimeStamp_StartFormat','wtc_AcWindSp_mean;4']].groupby('wtc_AcWindSp_mean;4',as_index=False).rolling('2H', on='TimeStamp_StartFormat').apply(get_dominant_freq)
    return df

In [17]:
turbine_data = {
    "T01": {"Latitude": 57.49921441, "Longitude": -3.086742896},
    "T02": {"Latitude": 57.49626574, "Longitude": -3.082817716},
    "T03": {"Latitude": 57.50206973, "Longitude": -3.088980479},
    "T04": {"Latitude": 57.50196736, "Longitude": -3.082085466},
    "T05": {"Latitude": 57.49891107, "Longitude": -3.078123820},
    "T06": {"Latitude": 57.50024464, "Longitude": -3.071321578},
    "T07": {"Latitude": 57.50513302, "Longitude": -3.085850762},
    "T08": {"Latitude": 57.50465629, "Longitude": -3.077542275},
    "T09": {"Latitude": 57.50827835, "Longitude": -3.082572740},
    "T10": {"Latitude": 57.50542459, "Longitude": -3.070655631},
    "T11": {"Latitude": 57.51184311, "Longitude": -3.080658605},
    "T12": {"Latitude": 57.51081569, "Longitude": -3.073684523},
    "T13": {"Latitude": 57.51666161, "Longitude": -3.078146735},
    "T14": {"Latitude": 57.51376046, "Longitude": -3.074939555},
    "T15": {"Latitude": 57.49941809, "Longitude": -3.062837374},
    "T16": {"Latitude": 57.50513916, "Longitude": -3.053340822},
    "T17": {"Latitude": 57.50662266, "Longitude": -3.047792556},
    "T18": {"Latitude": 57.50429684, "Longitude": -3.041183981},
    "T19": {"Latitude": 57.50859585, "Longitude": -3.041656952},
    "T20": {"Latitude": 57.51202849, "Longitude": -3.040436127},
    "T21": {"Latitude": 57.50978499, "Longitude": -3.034697307}
}

In [18]:
def weighted_wake_proximity(row, turbine_lat_lon, target_turbine="T01"):
    total_weight = 0
    for tid, coords in turbine_lat_lon.items():
        if tid == target_turbine:
            continue
        if row[f'upstream_{tid}']:
            distance = haversine(coords["Longitude"], coords["Latitude"],
                                turbine_lat_lon[target_turbine]["Longitude"],
                                turbine_lat_lon[target_turbine]["Latitude"])
            total_weight += 1 / (distance + 1e-6)  # Avoid division by zero
    return total_weight



In [19]:
def filter_wake_affected_turbines(
    df, 
    turbine_lat_lon, 
    turbine_id="T01", 
    wind_dir_col='ERA5_wind_direction_100m',
    wake_threshold_deg=10
):
    """
    Creates new wake status columns while preserving all original data.
    
    Args:
        df: Input DataFrame with turbine data columns
        turbine_lat_lon: Dict of {turbine_id: {"Latitude": y, "Longitude": x}}
        turbine_id: Reference turbine ID (default "T01")
        wind_dir_col: Column name for wind direction
        wake_threshold_deg: Wake angle threshold in degrees
        
    Returns:
        DataFrame with new wake status columns (*;waked and *;waking)
    """
    # Convert turbine_id to string
    turbine_id = str(turbine_id)
    
    # Wind direction vector
    wind_dir_rad = np.radians(df[wind_dir_col].values)
    wind_u, wind_v = np.cos(wind_dir_rad), np.sin(wind_dir_rad)
    
    # Reference turbine coordinates
    ref_lat = turbine_lat_lon[turbine_id]["Latitude"]
    ref_lon = turbine_lat_lon[turbine_id]["Longitude"]
    
    # Create new columns for wake status
    result_df = df.copy()
    
    for tid, coords in turbine_lat_lon.items():
        if tid == turbine_id:
            continue  # Skip reference turbine
            
        # Calculate direction vector
        dx = coords["Longitude"] - ref_lon
        dy = coords["Latitude"] - ref_lat
        dist = np.sqrt(dx**2 + dy**2)
        if dist == 0:
            continue
            
        # Normalized direction vector
        dx /= dist
        dy /= dist
        
        # Wake detection
        dot_product = dx * wind_u + dy * wind_v
        angle_diff = np.degrees(np.arccos(np.clip(dot_product, -1, 1)))
        is_waked = angle_diff <= wake_threshold_deg
        
        # Reverse wake detection
        dot_product_rev = -dx * wind_u + -dy * wind_v
        angle_diff_rev = np.degrees(np.arccos(np.clip(dot_product_rev, -1, 1)))
        is_waking = angle_diff_rev <= wake_threshold_deg
        
        # Add wake status columns for each variable
        for col in df.columns:
            if f';{tid[-1]}' in col:  # Match turbine number in column
                base_col = col.split(';')[0]
                result_df[f'{base_col};waked'] = is_waked.astype(int)
                result_df[f'{base_col};waking'] = is_waking.astype(int)
    
    return result_df

In [20]:
input_df= input_df[~input_df.target.isna()]

In [21]:
#pattern = re.compile(r'max|min')#max|min
#test_df = test_df.loc[:, ~test_df.columns.str.contains(pattern)]

In [22]:
#input_df = new_features(input_df)
#test_df= new_features(test_df)

In [23]:
def lagged_valid(df,timestamp_col='TimeStamp_StartFormat',wind_col="is_valid"):
        lag_30min = timedelta(minutes=30)
        lag_1hr = timedelta(minutes=60)
        wind_col
        lagged = df[[timestamp_col, wind_col]].copy()
        
        # Calculate 30min lag
        lagged['temp_30min'] = lagged[timestamp_col] + lag_30min
        lagged = lagged.rename(columns={wind_col: f'{wind_col}_lag30min'})
        df = pd.merge_asof(
            df.sort_values(timestamp_col),
            lagged[['temp_30min', f'{wind_col}_lag30min']].sort_values('temp_30min'),
            left_on=timestamp_col,
            right_on='temp_30min',
            direction='backward'
        ).drop(columns='temp_30min')
        
        # Calculate 1hr lag
        lagged['temp_1hr'] = lagged[timestamp_col] + lag_1hr
        lagged = lagged.rename(columns={f'{wind_col}_lag30min': f'{wind_col}_lag1hr'})
        df = pd.merge_asof(
            df.sort_values(timestamp_col),
            lagged[['temp_1hr', f'{wind_col}_lag1hr']].sort_values('temp_1hr'),
            left_on=timestamp_col,
            right_on='temp_1hr',
            direction='backward'
        ).drop(columns='temp_1hr')
    
        return df

    
   
        # Calculate 30min lag


In [24]:

from math import radians, sin, cos, sqrt, atan2, degrees

def calculate_relative_angle(wind_dir, turbine_dir):
    """Calculate smallest angle between wind and turbine direction"""
    angle_diff = abs(wind_dir - turbine_dir) % 360
    return min(angle_diff, 360 - angle_diff)

def identify_upstream_turbines(df, turbine_lat_lon, target_turbine="T01", wind_dir_col='ERA5_wind_direction_100m'):
    """
    Create mask of turbines upstream of target for each timestep
    Returns: DataFrame with boolean columns 'upstream_T##' for each turbine
    """
    target_lat = turbine_lat_lon[target_turbine]["Latitude"]
    target_lon = turbine_lat_lon[target_turbine]["Longitude"]
    
    upstream_mask = pd.DataFrame(index=df.index)
    
    for tid, coords in turbine_lat_lon.items():
        if tid == target_turbine:
            continue
            
        # Calculate direction from target to current turbine
        dx = coords["Longitude"] - target_lon
        dy = coords["Latitude"] - target_lat
        turbine_dir = np.degrees(np.arctan2(dy, dx)) % 360
        
        # Compare to wind direction
        angle_diffs = df[wind_dir_col].apply(
            lambda wd: calculate_relative_angle(wd, turbine_dir)
        )
        
        # Turbine is upstream if within ±90° of wind direction
        upstream_mask[f'upstream_{tid}'] = angle_diffs <= 45
        
    return upstream_mask

def jensen_wake_deficit(wind_speed, upstream_mask_row, turbine_lat_lon, target_turbine="T01", k=0.04, D_rotor=82):
    """
    Calculate combined wake deficit from all upstream turbines
    """
    target_coords = turbine_lat_lon[target_turbine]
    total_deficit = 0
    
    for tid, is_upstream in upstream_mask_row.items():
        if not is_upstream or not tid.startswith('upstream_'):
            continue
            
        tid = tid.replace('upstream_', '')  # Get original turbine ID
        turbine_coords = turbine_lat_lon[tid]
        
        # Distance between turbines (simplified)
        dx = turbine_coords["Longitude"] - target_coords["Longitude"]
        dy = turbine_coords["Latitude"] - target_coords["Latitude"]
        distance = 111.32 * sqrt(dx**2 + dy**2) * 1000  # Convert to meters
        
        # Jensen wake model
        D_wake = D_rotor + 2 * k * distance
        deficit = (1 - sqrt(1 - (D_rotor/D_wake)**2)) * wind_speed
        total_deficit += deficit
        
    return min(total_deficit, wind_speed)  # Can't lose more than all wind

def add_wake_metrics(df, turbine_lat_lon, target_turbine="T01", wind_speed_col='ERA5_wind_speed_100m'):
    """
    Main function to add wake metrics to DataFrame
    """
    df = df.copy()
    
    # 1. Identify upstream turbines
    upstream_mask = identify_upstream_turbines(df, turbine_lat_lon, target_turbine)
    df = pd.concat([df, upstream_mask], axis=1)
    
    # 2. Calculate wake deficit (Jensen model)
    df['T01_wake_deficit'] = df.apply(
        lambda row: jensen_wake_deficit(
            row[wind_speed_col],
            row[[c for c in upstream_mask.columns]],
            turbine_lat_lon,
            target_turbine
        ),
        axis=1
    )
    
    # 3. Calculate available wind speed
    df['T01_effective_ws'] = df[wind_speed_col] - df['T01_wake_deficit']
    
    
    
    return df

In [25]:
def calculate_relative_angle(wind_dir, turbine_dir):
    """Calculate smallest angle between wind and turbine direction"""
    angle_diff = abs(wind_dir - turbine_dir) % 360
    return min(angle_diff, 360 - angle_diff)

def haversine_distance(lat1, lon1, lat2, lon2):
    """Compute distance in meters between two lat/lon points using Haversine formula"""
    R = 6371000  # Earth radius in meters
    phi1, phi2 = radians(lat1), radians(lat2)
    dphi = radians(lat2 - lat1)
    dlambda = radians(lon2 - lon1)
    
    a = sin(dphi / 2)**2 + cos(phi1) * cos(phi2) * sin(dlambda / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

def identify_upstream_turbines(df, turbine_lat_lon, target_turbine="T01", wind_dir_col='ERA5_wind_direction_100m', wake_half_angle=15):
    """
    Create mask of turbines upstream of target within wake cone
    """
    target_lat = turbine_lat_lon[target_turbine]["Latitude"]
    target_lon = turbine_lat_lon[target_turbine]["Longitude"]
    
    upstream_mask = pd.DataFrame(index=df.index)
    
    for tid, coords in turbine_lat_lon.items():
        if tid == target_turbine:
            continue

        # Direction from target to this turbine (for alignment)
        dx = coords["Longitude"] - target_lon
        dy = coords["Latitude"] - target_lat
        turbine_dir = degrees(atan2(dy, dx)) % 360
        
        # Calculate angular difference between wind direction and this turbine direction
        angle_diffs = df[wind_dir_col].apply(
            lambda wd: calculate_relative_angle(wd, turbine_dir)
        )
        
        # Turbine is upstream only if it's within the wake cone
        upstream_mask[f'upstream_{tid}'] = angle_diffs <= wake_half_angle
        
    return upstream_mask

def jensen_wake_deficit(wind_speed, upstream_mask_row, turbine_lat_lon, target_turbine="T01", k=0.04, default_D_rotor=82):
    """
    Calculate combined wake deficit from upstream turbines using quadratic sum
    """
    target_coords = turbine_lat_lon[target_turbine]
    target_lat = target_coords["Latitude"]
    target_lon = target_coords["Longitude"]
    
    total_deficit_squared = 0
    
    for colname, is_upstream in upstream_mask_row.items():
        if not is_upstream or not colname.startswith('upstream_'):
            continue
            
        tid = colname.replace('upstream_', '')
        turbine_coords = turbine_lat_lon[tid]
        rotor_diameter = turbine_coords.get("Rotor_diameter", default_D_rotor)
        
        # Distance between turbines
        dist = haversine_distance(
            turbine_coords["Latitude"], turbine_coords["Longitude"],
            target_lat, target_lon
        )
        
        # Jensen model wake radius at downstream distance
        D_wake = rotor_diameter + 2 * k * dist
        
        # Wake deficit from this turbine
        if D_wake > 0:
            deficit = wind_speed * (1 - sqrt(1 - (rotor_diameter / D_wake) ** 2))
            total_deficit_squared += deficit ** 2

    total_deficit = sqrt(total_deficit_squared)
    return min(total_deficit, wind_speed)

def add_wake_metrics(df, turbine_lat_lon, target_turbine="T01", wind_speed_col='ERA5_wind_speed_100m', wind_dir_col='ERA5_wind_direction_100m'):
    """
    Main function to compute and add wake metrics to DataFrame
    """
    df = df.copy()
    
    # Step 1: Identify upstream turbines within wake cone
    upstream_mask = identify_upstream_turbines(df, turbine_lat_lon, target_turbine, wind_dir_col)
    df = pd.concat([df, upstream_mask], axis=1)
    
    # Step 2: Compute Jensen wake deficit
    upstream_cols = [c for c in upstream_mask.columns]
    df[f'{target_turbine}_wake_deficit'] = df.apply(
        lambda row: jensen_wake_deficit(
            row[wind_speed_col],
            row[upstream_cols],
            turbine_lat_lon,
            target_turbine
        ),
        axis=1
    )
    
    # Step 3: Compute effective wind speed
    df[f'{target_turbine}_effective_ws'] = df[wind_speed_col] - df[f'{target_turbine}_wake_deficit']
    
    return df

In [26]:
input_df= add_wake_metrics(input_df, turbine_data, target_turbine="T01")
test_df= add_wake_metrics(test_df, turbine_data, target_turbine="T01")

In [27]:
input_df = lagged_valid(input_df)
test_df= lagged_valid(test_df)

In [28]:
from math import radians, sin, cos, sqrt, asin

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great-circle distance (in meters) between two points 
    on the Earth's surface given their latitude/longitude in degrees.
    """
    # Convert degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    
    # Haversine formula
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    
    # Earth radius in meters (mean radius = 6371 km)
    r = 6371 * 1000 
    return c * r

In [29]:
#input_df['weighted_wake_proximity'] = input_df.apply(
#    lambda row: weighted_wake_proximity(row, turbine_data),
#    axis=1
#)
#test_df['weighted_wake_proximity'] = test_df.apply(
#    lambda row: weighted_wake_proximity(row, turbine_data),
#    axis=1
#)

In [30]:
input_df = add_lagged_wind_speeds(input_df)
test_df= add_lagged_wind_speeds(test_df)

In [31]:
#input_df = day_feature_adder(input_df)
#test_df= day_feature_adder(test_df)

In [32]:
#input_df = add_trig_transforms(input_df)
#test_df= add_trig_transforms(test_df)

In [33]:
#input_df = filter_wake_affected_turbines(input_df, turbine_data)
#test_df = filter_wake_affected_turbines(test_df, turbine_data)

In [34]:
#input_df = add_lagged_act_power(input_df)
#test_df= add_lagged_act_power(test_df)

In [35]:
input_df.head()

,TimeStamp_StartFormat,wtc_AcWindSp_mean;1,wtc_AcWindSp_mean;2,wtc_AcWindSp_mean;3,wtc_AcWindSp_mean;4,wtc_AcWindSp_mean;5,wtc_AcWindSp_mean;7,wtc_AcWindSp_min;1,wtc_AcWindSp_min;2,wtc_AcWindSp_min;3,wtc_AcWindSp_min;4,wtc_AcWindSp_min;5,wtc_AcWindSp_min;7,wtc_AcWindSp_max;1,wtc_AcWindSp_max;2,wtc_AcWindSp_max;3,wtc_AcWindSp_max;4,wtc_AcWindSp_max;5,wtc_AcWindSp_max;7,wtc_AcWindSp_stddev;1,wtc_AcWindSp_stddev;2,wtc_AcWindSp_stddev;3,wtc_AcWindSp_stddev;4,wtc_AcWindSp_stddev;5,wtc_AcWindSp_stddev;7,wtc_ScYawPos_mean;1,wtc_ScYawPos_mean;2,wtc_ScYawPos_mean;3,wtc_ScYawPos_mean;4,wtc_ScYawPos_mean;5,wtc_ScYawPos_mean;7,wtc_ScYawPos_min;1,wtc_ScYawPos_min;2,wtc_ScYawPos_min;3,wtc_ScYawPos_min;4,wtc_ScYawPos_min;5,wtc_ScYawPos_min;7,wtc_ScYawPos_max;1,wtc_ScYawPos_max;2,wtc_ScYawPos_max;3,wtc_ScYawPos_max;4,wtc_ScYawPos_max;5,wtc_ScYawPos_max;7,wtc_ScYawPos_stddev;1,wtc_ScYawPos_stddev;2,wtc_ScYawPos_stddev;3,wtc_ScYawPos_stddev;4,wtc_ScYawPos_stddev;5,wtc_ScYawPos_stddev;7,wtc_NacelPos_mean;1,wtc_NacelPos_mean;2,wtc_NacelPos_mean;3,wtc_NacelPos_mean;4,wtc_NacelPos_mean;5,wtc_NacelPos_mean;7,wtc_NacelPos_min;1,wtc_NacelPos_min;2,wtc_NacelPos_min;3,wtc_NacelPos_min;4,wtc_NacelPos_min;5,wtc_NacelPos_min;7,wtc_NacelPos_max;1,wtc_NacelPos_max;2,wtc_NacelPos_max;3,wtc_NacelPos_max;4,wtc_NacelPos_max;5,wtc_NacelPos_max;7,wtc_GenRpm_mean;1,wtc_GenRpm_mean;2,wtc_GenRpm_mean;3,wtc_GenRpm_mean;4,wtc_GenRpm_mean;5,wtc_GenRpm_mean;7,wtc_GenRpm_min;1,wtc_GenRpm_min;2,wtc_GenRpm_min;3,wtc_GenRpm_min;4,wtc_GenRpm_min;5,wtc_GenRpm_min;7,wtc_GenRpm_max;1,wtc_GenRpm_max;2,wtc_GenRpm_max;3,wtc_GenRpm_max;4,wtc_GenRpm_max;5,wtc_GenRpm_max;7,wtc_GenRpm_stddev;1,wtc_GenRpm_stddev;2,wtc_GenRpm_stddev;3,wtc_GenRpm_stddev;4,wtc_GenRpm_stddev;5,wtc_GenRpm_stddev;7,wtc_PitcPosA_mean;1,wtc_PitcPosA_mean;2,wtc_PitcPosA_mean;3,wtc_PitcPosA_mean;4,wtc_PitcPosA_mean;5,wtc_PitcPosA_mean;7,wtc_PitcPosA_min;1,wtc_PitcPosA_min;2,wtc_PitcPosA_min;3,wtc_PitcPosA_min;4,wtc_PitcPosA_min;5,wtc_PitcPosA_min;7,wtc_PitcPosA_max;1,wtc_PitcPosA_max;2,wtc_PitcPosA_max;3,wtc_PitcPosA_max;4,wtc_PitcPosA_max;5,wtc_PitcPosA_max;7,wtc_PitcPosA_stddev;1,wtc_PitcPosA_stddev;2,wtc_PitcPosA_stddev;3,wtc_PitcPosA_stddev;4,wtc_PitcPosA_stddev;5,wtc_PitcPosA_stddev;7,wtc_PitcPosB_mean;1,wtc_PitcPosB_mean;2,wtc_PitcPosB_mean;3,wtc_PitcPosB_mean;4,wtc_PitcPosB_mean;5,wtc_PitcPosB_mean;7,wtc_PitcPosC_mean;1,wtc_PitcPosC_mean;2,wtc_PitcPosC_mean;3,wtc_PitcPosC_mean;4,wtc_PitcPosC_mean;5,wtc_PitcPosC_mean;7,wtc_PowerRef_endvalue;1,wtc_PowerRef_endvalue;2,wtc_PowerRef_endvalue;3,wtc_PowerRef_endvalue;4,wtc_PowerRef_endvalue;5,wtc_PowerRef_endvalue;7,wtc_ScReToOp_timeon;1,wtc_ScReToOp_timeon;2,wtc_ScReToOp_timeon;3,wtc_ScReToOp_timeon;4,wtc_ScReToOp_timeon;5,wtc_ScReToOp_timeon;7,wtc_ActPower_mean;1,wtc_ActPower_mean;2,wtc_ActPower_mean;3,wtc_ActPower_mean;4,wtc_ActPower_mean;5,wtc_ActPower_mean;7,wtc_ActPower_min;1,wtc_ActPower_min;2,wtc_ActPower_min;3,wtc_ActPower_min;4,wtc_ActPower_min;5,wtc_ActPower_min;7,wtc_ActPower_max;1,wtc_ActPower_max;2,wtc_ActPower_max;3,wtc_ActPower_max;4,wtc_ActPower_max;5,wtc_ActPower_max;7,wtc_ActPower_stddev;1,wtc_ActPower_stddev;2,wtc_ActPower_stddev;3,wtc_ActPower_stddev;4,wtc_ActPower_stddev;5,wtc_ActPower_stddev;7,wtc_AmbieTmp_mean;1,wtc_AmbieTmp_mean;2,wtc_AmbieTmp_mean;3,wtc_AmbieTmp_mean;4,wtc_AmbieTmp_mean;5,wtc_AmbieTmp_mean;7,ShutdownDuration;1,ShutdownDuration;2,ShutdownDuration;3,ShutdownDuration;4,ShutdownDuration;5,ShutdownDuration;7,ERA5_temperature_2m,ERA5_relative_humidity_2m,ERA5_dew_point_2m,ERA5_precipitation,ERA5_surface_pressure,ERA5_cloud_cover,ERA5_wind_speed_10m,ERA5_wind_speed_100m,ERA5_wind_direction_10m,ERA5_wind_direction_100m,ERA5_wind_gusts_10m,id,is_valid,target,hour,upstream_T02,upstream_T03,upstream_T04,upstream_T05,upstream_T06,upstream_T07,upstream_T08,upstream_T09,upstream_T10,upstream_T11,upstream_T12,upstream_T13,upstream_T14,upstream_T15,upstream_T16,upstream_T17,upstream_T18,upstream_T19,upstream_T20,upstream_T21,T01_wake_de

In [36]:
y = input_df["target"]


In [37]:
#input_df.replace([np.inf, -np.inf], np.nan, inplace=True)
#test_df = test_df.loc[:, ~test_df.columns.str.endswith("4")|~test_df.columns.str.endswith("5")|~test_df.columns.str.endswith("7")]
X_raw = input_df[test_df.columns].fillna(0).drop(["TimeStamp_StartFormat"],axis=1)
scaler = StandardScaler()
model = scaler.fit(X_raw)
X_scaled = model.transform(X_raw)

test_raw = test_df.fillna(0).drop(["TimeStamp_StartFormat"],axis=1)
#scaler = StandardScaler()
X_test = model.transform(test_raw)

In [38]:
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y, test_size=0.25, random_state=123
)


In [39]:
train_data = pd.DataFrame(X_train,columns = X_raw.columns).copy()
train_data['target'] = y_train

In [40]:
mask = np.isfinite(y_train)

# Step 2: Filter X_train and y_train accordingly
X_train_clean = X_train[mask]
y_train_clean = y_train[mask]

In [41]:
X_raw = input_df[test_df.columns].fillna(0).drop(["TimeStamp_StartFormat"],axis=1)
X_raw["target"] = input_df["target"]
test_raw = test_df.fillna(0).drop(["TimeStamp_StartFormat"],axis=1)


In [43]:
train_data = X_raw.copy()#pd.DataFrame(X_train_clean,columns = X_raw.columns).copy()
#train_data['target'] = y_train_clean
train_data = train_data[~train_data['target'].isna()]

In [44]:
predictor = TabularPredictor(
    label='target',
    problem_type='regression',
    eval_metric='mae'
).fit(
    train_data,
	presets="best",
    time_limit=3600*8
)

# Predict on validation set
#val_pred = predictor.predict(val_data)
#val_mae = mean_absolute_error(val_data["target"], val_pred)
#print(f"Validation MAE: {val_mae:.4f}")

# Predict on test set
test_pred = predictor.predict(test_raw)

# Save predictions
np.savetxt("predictions_autogluon_xgb.csv", test_pred, delimiter=",")

No path specified. Models will be saved in: "AutogluonModels\ag-20250718_170627"
Preset alias specified: 'best' maps to 'best_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.9
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
Memory Avail:       20.20 GB / 31.86 GB (63.4%)
Disk Space Avail:   1566.28 GB / 1863.01 GB (84.1%)
Presets specified: ['best']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGlu

(_ray_fit pid=3504) [1000]	valid_set's l1: 51.2086
(_ray_fit pid=3504) [2000]	valid_set's l1: 49.7179 [repeated 8x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(_ray_fit pid=9384) [3000]	valid_set's l1: 49.4093 [repeated 8x across cluster]
(_ray_fit pid=9384) [4000]	valid_set's l1: 48.9148 [repeated 8x across cluster]
(_ray_fit pid=9384) [5000]	valid_set's l1: 48.5847 [repeated 8x across cluster]
(_ray_fit pid=6016) [5000]	valid_set's l1: 48.1544 [repeated 5x across cluster]
(_ray_fit pid=9384) [6000]	valid_set's l1: 48.3474 [repeated 3x across cluster]
(_ray_fit pid=15288) [7000]	valid_set's l1: 48.0816 [repeated 8x across cluster]
(_ray_fit pid=11960) [7000]	valid_set's l1: 47.8154 [repeated 7x across cluster]
(_ray_fit pid=15288) [8000]	valid_set's l1: 47.9664
(_ray_fit pid=9384) [8000]	valid_set's l1:

(_dystack pid=20068) 	-47.6452	 = Validation score   (-mean_absolute_error)
(_dystack pid=20068) 	1085.36s	 = Training   runtime
(_dystack pid=20068) 	644.92s	 = Validation runtime
(_dystack pid=20068) Fitting model: LightGBM_BAG_L1 ... Training model for up to 3428.19s of the 5823.87s of remaining time.
(_dystack pid=20068) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=7.66%)


(_ray_fit pid=21024) [1000]	valid_set's l1: 50.6881 [repeated 4x across cluster]
(_ray_fit pid=1804) [2000]	valid_set's l1: 49.1929 [repeated 8x across cluster]
(_ray_fit pid=1804) [3000]	valid_set's l1: 48.5963 [repeated 8x across cluster]
(_ray_fit pid=21024) [4000]	valid_set's l1: 48.5698 [repeated 8x across cluster]
(_ray_fit pid=20192) [5000]	valid_set's l1: 48.9066 [repeated 8x across cluster]
(_ray_fit pid=20192) [6000]	valid_set's l1: 48.7338 [repeated 8x across cluster]
(_ray_fit pid=20192) [7000]	valid_set's l1: 48.6089 [repeated 8x across cluster]
(_ray_fit pid=20192) [8000]	valid_set's l1: 48.5323 [repeated 8x across cluster]
(_ray_fit pid=8820) [8000]	valid_set's l1: 48.0539 [repeated 7x across cluster]
(_ray_fit pid=20192) [9000]	valid_set's l1: 48.4751
(_ray_fit pid=5652) [9000]	valid_set's l1: 48.2618
(_ray_fit pid=20192) [10000]	valid_set's l1: 48.4333 [repeated 7x across cluster]
(_ray_fit pid=2852) [10000]	valid_set's l1: 48.1503 [repeated 7x across cluster]


(_dystack pid=20068) 	-48.0518	 = Validation score   (-mean_absolute_error)
(_dystack pid=20068) 	1156.9s	 = Training   runtime
(_dystack pid=20068) 	641.62s	 = Validation runtime
(_dystack pid=20068) Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 2182.98s of the 4578.65s of remaining time.
(_dystack pid=20068) d:\Renewables\towie\.venv\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
(_dystack pid=20068)   warnings.warn(
(_dystack pid=20068) 	-54.1846	 = Validation score   (-mean_absolute_error)
(_dystack pid=20068) 	1610.45s	 = Training   runtime
(_dystack pid=20068) 	21.97s	 = Validation runtime
(_dystack pid=20068) Fitting model: CatBoost_BAG_L1 ... Training model for up to 548.77s of the 2944.45s of remaining time.
(_dystack pid=20068) 	Memory not en

(_ray_fit pid=15108) [1000]	valid_set's l1: 47.7392
(_ray_fit pid=16792) [1000]	valid_set's l1: 46.7568
(_ray_fit pid=16792) [2000]	valid_set's l1: 46.587 [repeated 7x across cluster]
(_ray_fit pid=14888) [3000]	valid_set's l1: 46.6253 [repeated 8x across cluster]
(_ray_fit pid=15108) [4000]	valid_set's l1: 47.5124 [repeated 5x across cluster]
(_ray_fit pid=15108) [5000]	valid_set's l1: 47.5393 [repeated 2x across cluster]
(_ray_fit pid=11516) [6000]	valid_set's l1: 46.9307 [repeated 2x across cluster]
(_ray_fit pid=11516) [7000]	valid_set's l1: 46.9616


(_dystack pid=20068) 	-46.7345	 = Validation score   (-mean_absolute_error)
(_dystack pid=20068) 	588.47s	 = Training   runtime
(_dystack pid=20068) 	36.68s	 = Validation runtime
(_dystack pid=20068) Fitting model: LightGBM_BAG_L2 ... Training model for up to 1447.33s of the 1447.25s of remaining time.
(_dystack pid=20068) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (8 workers, per: cpus=1, gpus=0, memory=6.96%)
(_dystack pid=20068) 	-46.3063	 = Validation score   (-mean_absolute_error)
(_dystack pid=20068) 	58.35s	 = Training   runtime
(_dystack pid=20068) 	2.8s	 = Validation runtime
(_dystack pid=20068) Fitting model: RandomForestMSE_BAG_L2 ... Training model for up to 1382.45s of the 1382.37s of remaining time.
(_dystack pid=20068) d:\Renewables\towie\.venv\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead.

(raylet) The node with node id: f340e74e9fd7e108cd26b94ce37c9d5953c8debd34567d9315c4d6d1 and address: 127.0.0.1 and node name: 127.0.0.1 has been marked dead because the detector has missed too many heartbeats from it. This can happen when a 	(1) raylet crashes unexpectedly (OOM, etc.) 
	(2) raylet has lagging heartbeats due to slow network or busy workload.


In [45]:

# Prepare data as DataFrames with column names
train_data = X_raw.copy()#pd.DataFrame(X_train_clean,columns = X_raw.columns).copy()
#train_data['target'] = y_train_clean
train_data = train_data[~train_data['target'].isna()]
#val_data = pd.DataFrame(X_val,columns = X_raw.columns).copy()
#val_data['target'] = y_val
#val_data = val_data[~val_data['target'].isna()]
# Initialize predictor
predictor = TabularPredictor(
    label='target',
    problem_type='regression',
    eval_metric='mae'
).fit(
    train_data=train_data,
    #tuning_data=val_data,
    presets='experimental',
    #bagging=False,           # ✅ No Ray
    #num_bag_folds=0,         # ✅ Double confirm
    #use_bag_holdout=True,
    time_limit=3600*8,
    num_bag_folds=11, # to avoid raylets
    num_stack_levels=2 # when using bags is 2
     # Optional: increase if you want more output
)

# Predict on validation set
#val_pred = predictor.predict(val_data)
#val_mae = mean_absolute_error(val_data["target"], val_pred)
#print(f"Validation MAE: {val_mae:.4f}")

# Predict on test set
test_pred = predictor.predict(test_raw)

# Save predictions
np.savetxt("predictions_autogluon_xgb.csv", test_pred, delimiter=",")


No path specified. Models will be saved in: "AutogluonModels\ag-20250719_021226"
Preset alias specified: 'experimental' maps to 'experimental_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.9
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
Memory Avail:       20.07 GB / 31.86 GB (63.0%)
Disk Space Avail:   1563.23 GB / 1863.01 GB (83.9%)
Presets specified: ['experimental']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=2, num_bag_folds=11, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels`

LocalRayletDiedError: The task's local raylet died. Check raylet.out for more information.

In [ ]:
input_df.head()

In [ ]:
print(predictor.leaderboard(silent=True))

In [ ]:
predictor.feature_metadata.get_features()

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

# Load data
#train_data = TabularDataset('train.csv')
#test_data = TabularDataset('test.csv')

# Train AutoGluon (automates everything)
predictor = TabularPredictor(label='target', problem_type='regression').fit(
    train_data,
    time_limit=3600,
    presets='best_quality'
)

# Predict and evaluate
predictions = predictor.predict(test_data)
leaderboard = predictor.leaderboard(test_data)
print(leaderboard)

In [ ]:
xgb_model = xgb.XGBRegressor(
    objective='reg:absoluteerror',
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=12
    ,
    min_child_weight=1,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=0,
    reg_alpha=0,
    reg_lambda=1,
    random_state=42,
    n_jobs=-1
)

# Train the model
xgb_model.fit(
    X_train, 
    y_train,
    eval_set=[(X_val, y_val)],
    
    verbose=True
)

# Evaluate on validation set
val_pred = xgb_model.predict(X_val)
val_mae = mean_absolute_error(y_val, val_pred)
print(f"Validation MAE: {val_mae:.4f}")


# Make predictions
test_pred = xgb_model.predict(X_test)

# Save predictions
np.savetxt("predictions_xgboost.csv", test_pred, delimiter=",")

In [ ]:
#best 50.14 depth 12

In [ ]:

# Train basic model
model = xgb.XGBRegressor().fit(X_train, y_train)

# Visualize feature importance
xgb.plot_importance(model, max_num_features=20)
plt.show()

In [ ]:
from sklearn.feature_selection import mutual_info_regression

def evaluate_features(X, y):
    """Calculate mutual information scores"""
    mi_scores = mutual_info_regression(X, y)
    return pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)

# Usage:
mi_scores = evaluate_features(X_train, y_train)
print(mi_scores.head(20)) 